In [1]:
import numpy as np
from matplotlib import pyplot as plt, lines as mpllines
from scipy.constants import c as C
plt.rcParams.update({"xtick.direction": "in", "ytick.direction": "in", "font.size": 24})

In [2]:
def gaussian(x, center, FWHM, normalisation_coeffient = None):
    sigma = (8 *np.log(2))**-0.5 *FWHM
    exponent = -(1/2) *(x -center)**2 /(sigma**2)
    if normalisation_coeffient == None:
        normalisation_coeffient = 0.5/np.sum(np.abs(np.exp(exponent))) #1 /(sigma *(2*np.pi)**0.5)
        return normalisation_coeffient *np.exp(exponent), normalisation_coeffient
    else:
        return normalisation_coeffient *np.exp(exponent)

In [ ]:
array_length = 50 #in pixels
pixel_pitch = 1000 #in um
central_freq = 0.3 #in THz
FWHM_freq = 0.075 #in THz
theta = 366 #in arcminutes
signal_to_noise = np.inf #np.inf # maximum S:N
number_of_averages = 1 #Frames can be averaged to reduce UNCERTAINTY but not error.

In [4]:
central_wavenumber = 2*np.sin(np.deg2rad(theta/60)) *(central_freq*1e12) /C #periodicity of the fringes as it appears on the camera in m^-1
FWHM_wavenumber = 2*np.sin(np.deg2rad(theta/60)) *(FWHM_freq*1e12) /C # in m^-1

print(1/central_wavenumber/(pixel_pitch*1e-6)) # effective wavelength in pixels


4.780076938348003


In [ ]:
####### CREATE GAUSSIAN FREQUENCIES ###########

wavenumbers = np.fft.rfftfreq(array_length, (pixel_pitch*1e-6))
freqs = C *wavenumbers /(2*np.sin(np.deg2rad(theta/60))) *1e-12 #in THz
amplitudes, normalisation_coeffient = gaussian(wavenumbers, central_wavenumber, FWHM_wavenumber)
#amplitudes = phase *amplitudes
intensity = np.fft.irfft(amplitudes, norm= "forward", n= array_length)

oversampled_wn = np.fft.rfftfreq(array_length *10, (pixel_pitch*1e-6))
oversampled_freqs = C *oversampled_wn /(2*np.sin(np.deg2rad(theta/60))) *1e-12 #in THz
oversampled_amps = gaussian(oversampled_wn, central_wavenumber, FWHM_wavenumber, normalisation_coeffient= normalisation_coeffient)

displacement = np.arange(-(array_length//2), (array_length+1)//2) *pixel_pitch*1e-3 #in mm

if True: #toggle gunn diode double peak
    kappa = 2*np.sin(np.deg2rad(theta/60))/C * 273e9 # apparent wavenumber of the 273 GHz peak
    intensity = np.cos(2*np.pi*displacement*1e-3 *kappa)
    kappa = 2*np.sin(np.deg2rad(theta/60))/C * 182e9 # apparent wavenumber of the 182 GHz peak
    intensity += np.cos(2*np.pi*displacement*1e-3 *kappa)
    envelope_FWHM = 35 # mm gaussian beam size
    intensity *= np.exp(-4*np.log(2)*displacement**2 *envelope_FWHM**-2) # modulate by the beam size.
    intensity = np.fft.ifftshift(intensity)
    amplitudes = np.fft.rfft(intensity, norm= "forward") #overwrite amplitudes for 270GHz source

displacement


array([-25., -24., -23., -22., -21., -20., -19., -18., -17., -16., -15.,
       -14., -13., -12., -11., -10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,
        -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,
         8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,
        19.,  20.,  21.,  22.,  23.,  24.])

In [6]:
####### ADD NOISE DUE TO VIBRATIONS ########
vibration_uncertainty = 0 #mm. uncertainty of the position of the peak due to vibrations.

temporal_sigma = vibration_uncertainty*1e-3 *2*np.sin(np.deg2rad(theta/60)) /C # convert to s.
noise = np.random.normal(0, temporal_sigma, number_of_averages)

vibration_amps = np.repeat(amplitudes[None, :], number_of_averages, axis= 0)
print(vibration_amps.shape)
vibration_amps = (vibration_amps *np.exp(1j *freqs[None, :]*1e12 *noise[:, None]))

(1, 26)


In [7]:
####### IFFT TO MAKE INTERFEROGRAMS ###########

vibration_intensity = np.fft.irfft(vibration_amps, norm= "forward", axis= 1, n= array_length)

In [8]:
####### ADD READ NOISE ########

read_noise_sigma = 1/signal_to_noise # as a fraction of the interferogram peak signal. 
print(read_noise_sigma)

noise = np.random.normal(0, read_noise_sigma, (number_of_averages, array_length))
noisy_intensity = vibration_intensity +noise
noisy_amps = np.fft.rfft(noisy_intensity, norm= "forward")


0.05


In [9]:
########## 'TEMPORAL' AVERAGE ############
# noise = read_noise_sigma
# uncertainty = read_noise_sigma / sqrt(number_of_averages)

averaged_intensity = np.mean(noisy_intensity, axis= 0)

averaged_amps = np.fft.rfft(averaged_intensity, norm= "forward")

In [10]:
########## SHIFT ALL INTERFEROGRAMS AROUND 0 ############

intensity = np.fft.fftshift(intensity)
vibration_intensity = np.fft.fftshift(vibration_intensity, axes= 1)
noisy_intensity = np.fft.fftshift(noisy_intensity, axes= 1)
averaged_intensity = np.fft.fftshift(averaged_intensity)

In [11]:
fig, axs = plt.subplots(2,1, figsize= (12,10), gridspec_kw= {"hspace": 0.3})

axs[0].plot(oversampled_freqs, np.abs(oversampled_amps), color= "black", label= "Gaussian curve")
axs[0].plot(freqs, np.abs(amplitudes), linestyle= "None", marker= "o", markersize= 6, color= "k", label= "resolvable frequencies")
axs[0].set(xlabel= "Frequency (THz)", ylabel= "Amplitude (arb.)", xlim= (0, None))
axs[0].set_title("a)", loc= "left")
axs[0].grid()
axs[0].legend(loc= "upper right")

axs[1].plot(displacement, averaged_intensity, color= "k", marker= "o", label= "resolvable points")
errorfillbetween = axs[1].fill_between(displacement, averaged_intensity- 3*read_noise_sigma, averaged_intensity+ 3*read_noise_sigma, facecolor= "gray", alpha= 0.8, label= r"± 3$\sigma$")
axs[1].set(xlabel= "x displacement (mm)" , ylabel= "Intensity (arb.)", xlim= (displacement.min(), displacement.max()), ylim= (None, 2))
axs[1].set_title("b)", loc= "left")
axs[1].grid()

legend_elements = [mpllines.Line2D([0], [0], color= "black", marker= "o", linestyle= "none", label= "resolvable points"),
                   errorfillbetween]

axs[1].legend(handles= legend_elements, loc= "upper right")

plt.savefig(r"./graphs/example_simulated_spectrum")
plt.show()

In [12]:
########### SAVE AN INTERFEROGRAM FOR TESTING #############

error = np.ones_like(averaged_intensity) if signal_to_noise == np.inf else np.full_like(averaged_intensity, read_noise_sigma)

to_save = np.array([averaged_intensity, error]).T
np.savetxt("data\\1dmockanderrors31.csv", to_save, delimiter=",")